In [7]:
import gc
import os
import numpy as np
import pandas as pd
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import keras

In [8]:
data = pd.read_csv('train.csv').drop(columns=['date_id'])
data

,D1,D2,D3,D4,D5,D6,D7,D8,D9,E1,...,V3,V4,V5,V6,V7,V8,V9,forward_returns,risk_free_rate,market_forward_excess_returns
0,0,0,0,1,1,0,0,0,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.002421,0.000301,-0.003038
1,0,0,0,1,1,0,0,0,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.008495,0.000303,-0.009114
2,0,0,0,1,0,0,0,0,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.009624,0.000301,-0.010243
3,0,0,0,1,0,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004662,0.000299,0.004046
4,0,0,0,1,0,0,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.011686,0.000299,-0.012301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8985,0,0,0,0,0,0,0,0,0,1.565379,...,0.469577,0.837963,1.226772,0.822751,-0.707361,0.142857,-0.649616,0.002457,0.000155,0.001990
8986,0,0,0,0,0,0,0,0,0,1.562946,...,0.671958,0.837963,0.785877,0.805556,-0.715692,0.196098,-0.668289,0.002312,0.000156,0.001845
8987,0,0,1,0,0,0,0,0,0,1.560520,...,0.481481,0.787698,0.834898,0.823413,-0.723949,0.133929,-0.670946,0.002891,0.000156,0.002424
8988,0,0,0,0,0,0,0,0,0,1.558102,...,0.655423,0.783730,0.994026,0.851852,-0.684937,0.101852,-0.646265,0.008310,0.000156,0.007843


In [9]:
y = np.where(data['market_forward_excess_returns'] > 0, 2.0, 0.0)
y

array([0., 0., 0., ..., 2., 2., 0.], shape=(8990,))

In [10]:
X = data.drop(columns=['market_forward_excess_returns'])
for column in ['forward_returns', 'risk_free_rate']: # test data columns are lagged by 1 day
    X[column] = X[column].shift(1)
X = X.fillna(0)
X

,D1,D2,D3,D4,D5,D6,D7,D8,D9,E1,...,V2,V3,V4,V5,V6,V7,V8,V9,forward_returns,risk_free_rate
0,0,0,0,1,1,0,0,0,1,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0,0,0,1,1,0,0,0,1,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.002421,0.000301
2,0,0,0,1,0,0,0,0,1,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.008495,0.000303
3,0,0,0,1,0,0,0,0,0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.009624,0.000301
4,0,0,0,1,0,0,0,0,0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004662,0.000299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8985,0,0,0,0,0,0,0,0,0,1.565379,...,0.785053,0.469577,0.837963,1.226772,0.822751,-0.707361,0.142857,-0.649616,-0.002896,0.000159
8986,0,0,0,0,0,0,0,0,0,1.562946,...,0.767857,0.671958,0.837963,0.785877,0.805556,-0.715692,0.196098,-0.668289,0.002457,0.000155
8987,0,0,1,0,0,0,0,0,0,1.560520,...,0.734127,0.481481,0.787698,0.834898,0.823413,-0.723949,0.133929,-0.670946,0.002312,0.000156
8988,0,0,0,0,0,0,0,0,0,1.558102,...,0.695106,0.655423,0.783730,0.994026,0.851852,-0.684937,0.101852,-0.646265,0.002891,0.000156


In [11]:
n_features = len(X.columns) # 94

layers = keras.layers
Sequential = keras.Sequential

models = [
    Sequential([
        layers.Input([n_features]),
        layers.Dense(int(2 * n_features // 3)),
        layers.Dense(int(n_features // 3)),
        layers.Dense(1),
    ], name='v16'),
]

for model in models:
    model.compile(loss=keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam(), metrics=[keras.metrics.MeanAbsoluteError()])


In [12]:
n = len(X)
i_train = slice(0, int(n * 0.8))
i_val = slice(int(n * 0.8), None)
for i, model in enumerate(models):
    prefix = f'Training model {i+1}/{len(models)}: {model.name}'
    print(f'{prefix}...')
    model.fit(X.iloc[i_train, :], y[i_train], epochs=1000, validation_data=(X.iloc[i_val, :], y[i_val]), #verbose=0,
              callbacks=[keras.callbacks.TerminateOnNaN(), keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)])
    model.save(f'.models/{model.name}.keras')
    print(f'{prefix}...Complete.')
    while gc.collect() > 0: pass # clear as much memory as possible

Training model 1/1: v16...
Epoch 1/1000
225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 1.2055 - mean_absolute_error: 1.0081 - val_loss: 1.1481 - val_mean_absolute_error: 1.0138
Epoch 2/1000
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 780us/step - loss: 1.0499 - mean_absolute_error: 0.9915 - val_loss: 1.0928 - val_mean_absolute_error: 1.0024
Epoch 3/1000
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 857us/step - loss: 1.0249 - mean_absolute_error: 0.9880 - val_loss: 1.0335 - val_mean_absolute_error: 0.9945
Epoch 4/1000
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 846us/step - loss: 1.0204 - mean_absolute_error: 0.9898 - val_loss: 1.0421 - val_mean_absolute_error: 0.9937
Epoch 5/1000
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 911us/step - loss: 1.0190 - mean_absolute_error: 0.9888 - val_loss: 1.0707 - val_mean_absolute_error: 1.0021
Epoch 6/1000
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 875us/step - loss: 1.0149 - mean_absolute_error: 0.9895 - val_loss: 1.0827 - val_mean_absolute_error: 0.9928
Epoch 7/1000
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 832us/